In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree


import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn

In [ ]:
from google.colab import drive
drive.mount('/drive') #mount drive first then read data from Colab

for dirname, _, filenames in os.walk('/drive/MyDrive/Colab Notebooks'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train_data = pd.read_csv("/drive/MyDrive/Colab Notebooks/titanic/train.csv")
test_data = pd.read_csv("/drive/MyDrive/Colab Notebooks/titanic/test.csv")
train_data.head()
test_data.head()

In [ ]:
# xử lý cả 2 tập train và test
train_data['is_train'] = 1
test_data['is_train'] = 0

test_passenger_ids = test_data['PassengerId']

# Gộp 2 file lại
full_data = pd.concat([train_data, test_data], sort=False)


# Chuyển 'male' -> 0, 'female' -> 1
full_data['Sex'] = full_data['Sex'].map({'male': 0, 'female': 1})

# 2. Xử lý cột 'Age' (Dữ liệu thiếu - NaN)
mean_age = full_data['Age'].mean()
full_data['Age'] = full_data['Age'].fillna(mean_age)
print(f"\nĐiền tuổi bị thiếu bằng tuổi trung bình: {mean_age:.2f}")

# Tìm giá trị xuất hiện nhiều nhất (mode)
mode_embarked = full_data['Embarked'].mode()[0]
full_data['Embarked'] = full_data['Embarked'].fillna(mode_embarked)
# Chuyển dữ liệu chữ thành số (One-Hot Encoding)
full_data = pd.get_dummies(full_data, columns=['Embarked'], drop_first=True)

# Chọn các đặc trưng
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked_Q', 'Embarked_S']

# Tách dữ liệu trở lại thành train và test
train_df = full_data[full_data['is_train'] == 1]
test_df = full_data[full_data['is_train'] == 0]

# Tạo X (features) và y (target)
X = train_df[features]
y = train_df['Survived']

X_test_final = test_df[features]

print("\nDữ liệu sau khi đã xử lý (5 hàng đầu):")
print(X.head())

In [ ]:
# Khởi tạo mô hình Cây quyết định
clf = DecisionTreeClassifier(max_depth=3, random_state=42)

# Huấn luyện mô hình
clf.fit(X, y)

print("\nĐã huấn luyện xong mô hình!")

# Đánh giá nhanh trên chính tập huấn luyện (chỉ để tham khảo)
y_pred_train = clf.predict(X)
train_accuracy = accuracy_score(y, y_pred_train)
print(f"Độ chính xác trên tập huấn luyện: {train_accuracy * 100:.2f}%")

In [ ]:
plt.figure(figsize=(15, 10))
tree.plot_tree(clf,
               feature_names=features,
               class_names=['Not Survived', 'Survived'],
               filled=True,
               rounded=True)
plt.show()

In [ ]:
# Dự đoán trên tập X_test_final
predictions = clf.predict(X_test_final)

submission = pd.DataFrame({
    'PassengerId': test_passenger_ids,
    'Survived': predictions.astype(int)
})

# Xuất ra file CSV
submission.to_csv('submission.csv', index=False)

print("\nĐã tạo file submission.csv. Bạn có thể nộp file này lên Kaggle!")
print(submission.head())